In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from PIL import Image
from torch.utils.data import Dataset

**Load data**

In [ ]:
test_file = "./data/nyu2_test.csv"
train_file = "./data/nyu2_train.csv"

In [ ]:
df_train = pd.read_csv(train_file, names=["img", "depth"], header=None)
df_test = pd.read_csv(test_file, names=["img", "depth"], header=None)

In [ ]:
df_train.shape, df_test.shape

In [ ]:
50688 + 654

In [ ]:
df_train.head()

In [ ]:
df_test.head()

**Visualize example data**

In [ ]:
def visualize_example(img_path, depth_path):
    img = Image.open(img_path)
    depth = Image.open(depth_path)

    _, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 6))

    axes[0].imshow(img)
    axes[0].set_title("Image")

    axes[1].imshow(depth)
    axes[1].set_title("Depth")

    # hide the axis ticks
    for ax in axes:
        ax.axis("off")

    plt.show()

In [ ]:
img_path = df_train.iloc[0]["img"]
depth_path = df_train.iloc[0]["depth"]

visualize_example(img_path, depth_path)

In [ ]:
img = Image.open(img_path).convert("RGB")
np.array(img).shape

**Check mask values**

In [ ]:
train_mask = Image.open(df_train.iloc[0]["depth"])
train_mask = np.asarray(train_mask, np.int32)

In [ ]:
train_mask

In [ ]:
train_mask / 10_000

In [ ]:
def bilinear_interpolate(im, size):
    return F.interpolate(im, size=size, mode="bilinear", align_corners=True)

In [ ]:
res = (
    bilinear_interpolate(torch.tensor(train_mask).unsqueeze(0).unsqueeze(0).float(), size=(56, 56))
    .squeeze()
    .numpy()
)

In [ ]:
plt.imshow(train_mask)

In [ ]:
plt.imshow(res)

In [ ]:
test_mask = Image.open(df_test.iloc[0]["depth"])
test_mask = np.asarray(test_mask, np.int32)
test_mask

In [ ]:
test_mask / 10_000 * 255

**NYU Depth V2 Dataset**

In [ ]:
class NYUDataset(Dataset):
    def __init__(self, df, data_dir, transform=None, target_transform=None):
        self.df = df
        self.data_dir = data_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.data_dir, self.df.iloc[idx, 0])
        mask_path = os.path.join(self.data_dir, self.df.iloc[idx, 1])

        img = np.asarray(Image.open(img_path))
        mask = np.asarray(Image.open(mask_path))

        if self.transform:
            img = self.transform(img)
        if self.target_transform:
            mask = self.target_transform(mask)

        return img, mask

In [ ]:
class NormalizeData(object):
    def __init__(self, divisor):
        self.divisor = divisor

    def __call__(self, img):
        return img / self.divisor

In [ ]:
test_dataset = NYUDataset(df_test, "..", target_transform=NormalizeData(10_000))

In [ ]:
next(iter(test_dataset))[1]